In [ ]:
include("setup.jl")

using Trixi

function plot_eigenvalues(λ)
    fig, ax = plt.subplots(1, 1)
    ax.set_prop_cycle(marker_cycler)
    plt.plot(real.(λ), imag.(λ))
    plt.xlabel(L"\mathrm{Re}(\lambda)")
    plt.ylabel(L"\mathrm{Im}(\lambda)")
    
    fig
end

# Using a numerical flux with the logarithmic mean in the density results in negative densities

You need to wait some time for the first run because of compilation time.

In [ ]:
# numerical_flux = flux_central
# numerical_flux = flux_shima_etal
numerical_flux = flux_ranocha

Trixi.run("parameters_density_wave.toml", 
            volume_integral_type="split_form", cfl=0.05,
            surface_flux=numerical_flux, volume_flux=numerical_flux)

# The spectrum contains eigenvalues with positive real part for volume fluxes different from the central flux

In [ ]:
numerical_flux = flux_central
J_central = Trixi.compute_jacobian_dg("parameters_density_wave.toml", 
                    volume_integral_type="split_form", cfl=0.05,
                    surface_flux=numerical_flux, volume_flux=numerical_flux)
λ_central = eigvals(J_central)

numerical_flux = flux_shima_etal
J_shima_etal = Trixi.compute_jacobian_dg("parameters_density_wave.toml", 
                                volume_integral_type="split_form", cfl=0.05,
                                surface_flux=numerical_flux, volume_flux=numerical_flux)
λ_shima_etal = eigvals(J_shima_etal)

numerical_flux = flux_ranocha
J_ranocha = Trixi.compute_jacobian_dg("parameters_density_wave.toml", 
                                volume_integral_type="split_form", cfl=0.05,
                                surface_flux=numerical_flux, volume_flux=numerical_flux)
λ_ranocha = eigvals(J_ranocha)


In [ ]:
function modify!(fig)
    fig.axes[1].set_xlim(-2, 2)
    fig.axes[1].set_ylim(-1000, 1000)
    fig
end

fig_central = plot_eigenvalues(λ_central) |> modify!
fig_central.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_Euler_central.pdf"), bbox_inches="tight")
fig_shima_etal = plot_eigenvalues(λ_shima_etal) |> modify!
fig_shima_etal.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_Euler_shima_etal.pdf"), bbox_inches="tight")
fig_ranocha = plot_eigenvalues(λ_ranocha) |> modify!
fig_ranocha.savefig(joinpath(dirname(@__DIR__), "figures", "spectrum_Euler_ranocha.pdf"), bbox_inches="tight")

# Fluctuation simulation without dissipation

In [ ]:
kwargs = (volume_integral_type="split_form", cfl=0.05,
            surface_flux=flux_shima_etal, volume_flux=flux_shima_etal)
J_shima_etal = Trixi.compute_jacobian_dg("parameters_density_wave.toml"; kwargs...)
λv_shima_etal = eigen(J_shima_etal)


idx = findlast(λ -> real(λ) > 0 && abs2(imag(λ)) < 1.0e-10, λv_shima_etal.values)

@show λv_shima_etal.values[idx]
@show extrema(imag, λv_shima_etal.vectors[:, idx])

u = real.(λv_shima_etal.vectors[:, idx])
u = 1.0e-3 / maximum(abs, u) * u
u_fluctuation = reshape(u, 4, 6, 6, 16) # 4 variables, polydeg=5, polydeg=5, nelements=16

mesh, dg, time_parameters, time_integration_function = Trixi.init_simulation()
u0_baseline = copy(dg.elements.u)
Trixi.rhs!(dg, 0.0)
ut_baseline = copy(dg.elements.u_t);

In [ ]:
let ut_baseline=ut_baseline
    function source_terms_fluctuations(ut, u, x, element_id, t, n_nodes, equation::CompressibleEulerEquations2D)
      for j in 1:n_nodes, i in 1:n_nodes, v in 1:4
          ut[v, i, j, element_id] -= ut_baseline[v, i, j, element_id]
      end

      return nothing
    end
    
    function initial_conditions_fluctuations(x, t, equation::CompressibleEulerEquations2D)
      v1 = 0.1
      v2 = 0.2
#       rho = 1 + 0.98 * sinpi(2 * (x[1] + x[2] - t * (v1 + v2)))
      rho = 1 + 0.98 * sinpi(2 * (x[1] + x[2]))
      rho_v1 = rho * v1
      rho_v2 = rho * v2
      p = 20
      rho_e = p / (equation.gamma - 1) + 1/2 * rho * (v1^2 + v2^2)
      return SVector(rho, rho_v1, rho_v2, rho_e)
    end
    
    Trixi.reset_timer!(Trixi.timer())
    Trixi.init_parameters("parameters_density_wave.toml"; kwargs...,
                            cfl=0.05, t_end=10.0,
                            initial_conditions=initial_conditions_fluctuations,
                            source_terms=source_terms_fluctuations)
    mesh, dg, time_parameters, time_integration_function = Trixi.init_simulation()
    @. dg.elements.u = u0_baseline + u_fluctuation
    Trixi.run_simulation(mesh, dg, time_parameters, time_integration_function)
end


In [ ]:
data, header = readdlm(joinpath("out", "analysis.dat"), header=true)
t = data[:, 2]
l∞_ϱ   = data[:, 8]
l∞_ϱv1 = data[:, 9]
l∞_ϱv2 = data[:, 10]
l∞_ϱe  = data[:, 11]
l∞ = maximum(data[:, 8:11], dims=2) |> vec

fig, ax = plt.subplots(1, 1)
ax.plot(t, l∞, label="numerical experiment")
c1, c2 = fit_error_growth(t, l∞)
@show λv_shima_etal.values[idx]
@show extrema(imag, λv_shima_etal.vectors[:, idx])
@show c2 - λv_shima_etal.values[idx]
@show relative_error(c2, λv_shima_etal.values[idx])
c1 = 1.0e-3; c2 = real(λv_shima_etal.values[idx])
model = @. c1 * exp(c2 * t)
# ax.plot(t, model, label=@sprintf("\$%.5f \\exp(%.2f t)\$", c1, c2))
ax.plot(t, model, label=L"10^{-3} \exp(\lambda t)")
ax.set_xlabel(L"t")
ax.set_ylabel(L"$l^\infty$ error")
ax.set_yscale("log")
ax.legend()
fig.savefig(joinpath(dirname(@__DIR__), "figures", "Euler_fluctuations_shima_etal_twice.pdf"), bbox_inches="tight")

t[end]

# Fluctuation simulation: HLL surface flux

In [ ]:
kwargs = (volume_integral_type="split_form", cfl=0.05,
            surface_flux=flux_hll, volume_flux=flux_shima_etal)
J_shima_etal = Trixi.compute_jacobian_dg("parameters_density_wave.toml"; kwargs...)
λv_shima_etal = eigen(J_shima_etal)


idx = findlast(λ -> real(λ) > 0 && abs2(imag(λ)) < 1.0e-10, λv_shima_etal.values)

@show λv_shima_etal.values[idx]
@show extrema(imag, λv_shima_etal.vectors[:, idx])

u = real.(λv_shima_etal.vectors[:, idx])
u = 1.0e-3 / maximum(abs, u) * u
u_fluctuation = reshape(u, 4, 6, 6, 16)

mesh, dg, time_parameters, time_integration_function = Trixi.init_simulation()
u0_baseline = copy(dg.elements.u)
Trixi.rhs!(dg, 0.0)
ut_baseline = copy(dg.elements.u_t)

In [ ]:
let ut_baseline=ut_baseline
    function source_terms_fluctuations(ut, u, x, element_id, t, n_nodes, equation::CompressibleEulerEquations2D)
      for j in 1:n_nodes, i in 1:n_nodes, v in 1:4
          ut[v, i, j, element_id] -= ut_baseline[v, i, j, element_id]
      end

      return nothing
    end
    
    function initial_conditions_fluctuations(x, t, equation::CompressibleEulerEquations2D)
      v1 = 0.1
      v2 = 0.2
#       rho = 1 + 0.98 * sinpi(2 * (x[1] + x[2] - t * (v1 + v2)))
      rho = 1 + 0.98 * sinpi(2 * (x[1] + x[2]))
      rho_v1 = rho * v1
      rho_v2 = rho * v2
      p = 20
      rho_e = p / (equation.gamma - 1) + 1/2 * rho * (v1^2 + v2^2)
      return SVector(rho, rho_v1, rho_v2, rho_e)
    end
    
    Trixi.reset_timer!(Trixi.timer())
    Trixi.init_parameters("parameters_density_wave.toml"; kwargs...,
                            cfl=0.05, t_end=10.0,
                            initial_conditions=initial_conditions_fluctuations,
                            source_terms=source_terms_fluctuations)
    mesh, dg, time_parameters, time_integration_function = Trixi.init_simulation()
    @. dg.elements.u = u0_baseline + u_fluctuation
    Trixi.run_simulation(mesh, dg, time_parameters, time_integration_function)
end


In [ ]:
data, header = readdlm(joinpath("out", "analysis.dat"), header=true)
t = data[:, 2]
l∞_ϱ   = data[:, 8]
l∞_ϱv1 = data[:, 9]
l∞_ϱv2 = data[:, 10]
l∞_ϱe  = data[:, 11]
l∞ = maximum(data[:, 8:11], dims=2) |> vec

fig, ax = plt.subplots(1, 1)
ax.plot(t, l∞, label="numerical experiment")
c1, c2 = fit_error_growth(t, l∞)
@show λv_shima_etal.values[idx]
@show extrema(imag, λv_shima_etal.vectors[:, idx])
@show c2 - λv_shima_etal.values[idx]
@show relative_error(c2, λv_shima_etal.values[idx])
c1 = 1.0e-3; c2 = real(λv_shima_etal.values[idx])
model = @. c1 * exp(c2 * t)
# ax.plot(t, model, label=@sprintf("\$%.5f \\exp(%.2f t)\$", c1, c2))
ax.plot(t, model, label=L"10^{-3} \exp(\lambda t)")
ax.set_xlabel(L"t")
ax.set_ylabel(L"$l^\infty$ error")
ax.set_yscale("log")
ax.legend()
fig.savefig(joinpath(dirname(@__DIR__), "figures", "Euler_fluctuations_shima_etal_hll.pdf"), bbox_inches="tight")

t[end]